In [ ]:
import random
import numpy as np


class Function:
    def __init__(self, value, delt=0.001):
        self.value = value

        def grad(x):
            array = []
            for i in range(len(x)):
                x[i] += delt
                first_val = value(x)
                x[i] -= 2 * delt
                second_val = value(x)
                x[i] += delt
                array.append((first_val - second_val) / (2 * delt))
            return np.array(array)

        self.grad = grad


def grad_desc(func, lr, x, eps, need_points=True,
              max_num_of_step=1000):
    num_of_steps = 0
    points = [x]
    while True:
        prev_x = x
        grad = func.grad(x)
        x = x - lr(Function(lambda a: func.value(x - a * grad))) * grad
        if need_points:
            points.append(x)
        else:
            points = [x]
        num_of_steps += 1
        if abs(func.value(x) - func.value(prev_x)) < eps or num_of_steps >= max_num_of_step:
            break

    if need_points: return np.array(points)
    return (points[0], num_of_steps)


def right_border_calc(func):
    right_start = 0.0000001
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt):
    cond = lambda a, b: abs(a - b) >= eps
    while cond(a_1, a_2):
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt
        fv1 = func.value(new_a_1)
        fv2 = func.value(new_a_2)
        if fv2 > fv1:
            a_2 = new_a_2
        elif fv2 < fv1:
            a_1 = new_a_1
        else:
            a_1 = new_a_1
            a_2 = new_a_2
    return (a_1 + a_2) / 2

In [ ]:
import matplotlib.pyplot as plt

def lr_wrapper(eps, delt):
    return lambda func: dichotomy(func, 0, right_border_calc(func), eps, delt)